### Get Packages 

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

%run ml_helpers.ipynb
%run ml.ipynb

       Unnamed: 0  beat  beat_num  district  sector  Year Month   Watch  Beat  \
29557         108     1      1713        17       1  2018     1   First  1713   
29558         109     1      1713        17       1  2018     1  Second  1713   
29559         110     1      1713        17       1  2018     1   Third  1713   
29560         111     1      1713        17       1  2018     2   Third  1713   
29561         112     1      1713        17       1  2018     2   First  1713   
...           ...   ...       ...       ...     ...   ...   ...     ...   ...   
49256       59052     1       312         3       1  2019    11  Second   312   
49257       59053     1       312         3       1  2019    11   Third   312   
49258       59054     1       312         3       1  2019    12  Second   312   
49259       59055     1       312         3       1  2019    12   First   312   
49260       59056     1       312         3       1  2019    12   Third   312   

       Crimes  ...  count_r

C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Finished one-hot encoding...
Finished standardizing...
0    0.852368
1    0.852368
2    0.852368
3    0.852368
4    0.852368
Name: principal component 1, dtype: float64
(19695, 2) (19695, 328)
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...


C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.85269
1    0.85269
2    0.85269
3    0.85269
4    0.85269
Name: principal component 1, dtype: float64
(19696, 2) (19696, 328)
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data


...

Finished one-hot encoding...
Finished standardizing...
0    0.852368
1    0.852368
2    0.852368
3    0.852368
4    0.852368
Name: principal component 1, dtype: float64
(19695, 2) (19695, 328)
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...


C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Finished one-hot encoding...
Finished standardizing...
0    0.85269
1    0.85269
2    0.85269
3    0.85269
4    0.85269
Name: principal component 1, dtype: float64
(19696, 2) (19696, 328)
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...


C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-bc70820a9afc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"]=df["Year"].astype("category")
C:\Users\dcarb\anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
0    0.856576
1    0.856576
2    0.856576
3    0.856576
4    0.856576
Name: principal component 1, dtype: float64
(19712, 2) (19712, 328)
         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1116433     1116433  1013  2019      3  Second  0.00   0.0    49    26   
1243410     1243410  2024  2019      8  Second  0.00   0.0    81    66   
1290020     1290020  1134  2019     10  Second  1.07   1.4    42    33   
1301521     1301521  1925  2019     11   Third  0.00   0.0    42    29   
1295023     1295023  2232  2019     11   Third  0.00   0.0    58    31   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1266237     1266237  2524  2019      9  Second  0.00   0.0    73    57   
1177571     1177571   421  2019      6  Second  0.34   0.0    86    60   
1174230     1174230  1013  2019      6   First  0.21   0.0    82    54   
1207096     1207096  1732  2019      7

In [2]:
#Get Final data: 
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest"])

data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)
data_list_20 = prep_data_test(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", "Beat", "Watch"])
data.dtypes

      district sector  Year Month   Watch  Beat  Serious  Domestic       TMAX  \
29557       17      1  2018     1   First  1713      1.0         2  33.201426   
29558       17      1  2018     1  Second  1713      8.0         4  33.201426   
29559       17      1  2018     1   Third  1713     11.0         4  33.201426   
29560       17      1  2018     2   Third  1713      5.0         3  39.791635   
29561       17      1  2018     2   First  1713      6.0         2  39.791635   
...        ...    ...   ...   ...     ...   ...      ...       ...        ...   
49256        3      1  2019    11  Second   312     13.0        11  43.761013   
49257        3      1  2019    11   Third   312     13.0         8  43.761013   
49258        3      1  2019    12  Second   312     20.0         8  42.647269   
49259        3      1  2019    12   First   312      7.0         8  42.647269   
49260        3      1  2019    12   Third   312     13.0         4  42.647269   

            TMIN  ...  coun

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


district               category
sector                 category
Year                      int64
Month                  category
Watch                  category
Beat                   category
Serious                 float64
Domestic                  int64
TMAX                    float64
TMIN                    float64
PRCP                    float64
SNOW                    float64
high_crime             category
count_l_stops           float64
count_bus_stops         float64
count_metra_stops       float64
count_restaurants       float64
count_bars              float64
count_daycares          float64
count_entertainment     float64
count_businesses        float64
road_distance_ft        float64
TOTAL POPULATION        float64
dist_to_police          float64
dist_to_hospital        float64
high_crime_geog_lag     float64
dtype: object

In [3]:
data_list_20[0][0]

,Year,Serious,Domestic,TMAX,TMIN,PRCP,SNOW,high_crime,count_l_stops,count_bus_stops,...,Beat_2524,Beat_2525,Beat_2531,Beat_2532,Beat_2533,Beat_2534,Beat_2535,Watch_First,Watch_Second,Watch_Third
29557,2018,-1.254018,-0.644941,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,1,0,0
29558,2018,-0.620498,-0.112728,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,1,0
29559,2018,-0.348990,-0.112728,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,0,1
29560,2018,-0.892007,-0.378835,-1.034854,-1.191190,0.608665,3.009861,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,0,1
29561,2018,-0.801504,-0.644941,-1.034854,-1.191190,0.608665,3.009861,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49256,2019,-0.167984,1.750018,-0.829309,-0.790072,-1.045229,-0.115556,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,1,0
49257,2019,-0.167984,0.951698,-0.829309,-0.790072,-1.045229,-0.115556,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,0,1
49258,2019,0.465536,0.951698,-0.886981,-0.889857,-1.225095,-0.297459,1.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,1,0
49259,2019,-0.711001,0.951698,-0.886981,-0.889857,-1.225095,-0.297459,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,1,0,0


In [4]:
def compute_scores_final_data(grid, model, model_name, data_list, target, target_year):
    '''
    Function that computes dataframe for results of best models
    Inputs:
        grid (dict): maps the model_name to a list which contains a dictionary mapping parameter (keys) to their 
                     values in the gest model
        model (sklearn): An sklearn algorithm
        model_name: a string assigned to the model, typically the same as model without the last parentheses
        data_list: list of tuples of dataframes. IN THIS CASE, this should include for 2020
        target (string): the column we seek to predict in the data frames
        target_year (int): 2020, the year we seek to predict
    Output:
        results (df): a dataframe with the results
        model (sklearn object): the model object for the predictor of 2020 data
        X_test_20 (df): the 2020 testing data (useful for column names)
    
    Note: the code that removes the params columns comes from here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe
    '''
    results, model, X_test_20 = results_by_year(model, 
                                    model_name, grid, 
                                    data_list, 
                                    target, 
                                    target_year)
    results["model_type"]= model_name
    results["params"]=results["parameters_2020"]
    
    bool_array = results.columns.str.startswith("parameters")
    results2 = results.loc[:,~bool_array]
    return results2, model, X_test_20

## Logistic Regression

In [5]:
#Logistic Regression!
grid_log_reg ={"LogisticRegression": [{'penalty': 'l2',
                              'C': 0.001,
                              'random_state': 0}]}

In [6]:
results_log, model, X_test = compute_scores_final_data(grid_log_reg, LogisticRegression(),
                                    "LogisticRegression", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.9902173913043478,) |recall is: (0.3772256728778468,)
precision is: (0.987597911227154,) |recall is: (0.6011124354390147,)
precision is: (0.9846968805179518,) |recall is: (0.6765062676910635,)
precision is: (0.9652265542676501,) |recall is: (0.7511275112751128,)


In [7]:
results_log

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.990217,0.377226,0.987598,0.601112,0.984697,0.676506,0.965227,0.751128,LogisticRegression,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}"


In [8]:
#Below code to make model coefficient df from Stack Exchange: https://stackoverflow.com/questions/26951880/scikit-learn-linear-regression-how-to-get-coefficients-respective-features
model_coefs = pd.concat([pd.DataFrame(X_test.columns), 
                         pd.DataFrame(np.transpose(model.coef_))], axis = 1)
model_coefs.columns = ["Predictor", "Coefficient"]
model_coefs["Coefficient"]=model_coefs["Coefficient"].abs()
model_coefs.sort_values(by="Coefficient", ascending=False, inplace=True)
model_coefs.tail(50)

,Predictor,Coefficient
252,Beat_1654,0.001377
89,Beat_322,0.001374
58,Beat_111,0.001369
76,Beat_223,0.001344
51,Month_6,0.001334
273,Beat_1833,0.001325
251,Beat_1653,0.001285
209,Beat_1224,0.001265
322,Beat_2521,0.001164
287,Beat_1933,0.001160


## KNN

In [9]:
#KNN!
grid_knn = {"KNeighborsClassifier": [{'weights': 'uniform', 'n_neighbors': 20}]}
results_knn = compute_scores_final_data(grid_knn, KNeighborsClassifier(),
                                    "KNeighborsClassifier", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.8411811652035116,) |recall is: (0.43643892339544516,)
precision is: (0.8785399622404028,) |recall is: (0.5546285260230434,)
precision is: (0.896551724137931,) |recall is: (0.5677315002021835,)
precision is: (0.8852459016393442,) |recall is: (0.6199261992619927,)


In [10]:
results_knn[0]

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.841181,0.436439,0.87854,0.554629,0.896552,0.567732,0.885246,0.619926,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 20}"


## SVM: 

In [11]:
grid_svm = {'LinearSVC': [{'C': 0.01, 'random_state': 0}]}
results_svm = compute_scores_final_data(grid_svm, LinearSVC(), 'LinearSVC', data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.9975550122249389,) |recall is: (0.168944099378882,)
precision is: (0.9753615425816818,) |recall is: (0.7234803337306317,)
precision is: (1.0,) |recall is: (0.39547108774767487,)
precision is: (1.0,) |recall is: (0.24600246002460024,)


In [12]:
results_svm[0]

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.997555,0.168944,0.975362,0.72348,1.0,0.395471,1.0,0.246002,LinearSVC,"{'C': 0.01, 'random_state': 0}"


## Naive Bayes: 

In [13]:
grid_nb = {'GaussianNB': [{'priors': None}]}
results_nb = compute_scores_final_data(grid_nb, GaussianNB(), 'GaussianNB', data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.35992701940620336,) |recall is: (0.8985507246376812,)
precision is: (0.3835683033543996,) |recall is: (0.9404052443384983,)
precision is: (0.36359404096834264,) |recall is: (0.9474322684997978,)
precision is: (0.35583488516449413,) |recall is: (0.940139401394014,)


In [14]:
results_nb[0]

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.359927,0.898551,0.383568,0.940405,0.363594,0.947432,0.355835,0.940139,GaussianNB,{'priors': None}


## Decision Tree:

In [16]:
grid_dt = {'DecisionTreeClassifier': [{'criterion': 'gini', 'max_depth':1, 'min_samples_split':2, 'random_state':0}]}
results_dt = compute_scores_final_data(grid_dt, DecisionTreeClassifier(), 'DecisionTreeClassifier', data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (1.0,) |recall is: (0.4848861283643892,)
precision is: (1.0,) |recall is: (0.7477155343663091,)
precision is: (0.97918637653737,) |recall is: (0.8370400323493732,)
precision is: (0.9123568890643505,) |recall is: (0.947519475194752,)


In [20]:
results_dt[0]

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,1.0,0.484886,1.0,0.747716,0.979186,0.83704,0.912357,0.947519,DecisionTreeClassifier,"{'criterion': 'gini', 'max_depth': 1, 'min_sam..."


## Random Forest:

In [17]:
grid_rf = {'RandomForestClassifier': [{'n_estimators': 600, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2, 'random_state': 0}]}
results_rf = compute_scores_final_data(grid_rf, RandomForestClassifier(), 'RandomForestClassifier', data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.9907407407407407,) |recall is: (0.22153209109730848,)
precision is: (1.0,) |recall is: (0.39451728247914186,)
precision is: (0.9937611408199644,) |recall is: (0.45086938940558025,)
precision is: (0.9794275491949911,) |recall is: (0.4489544895448955,)


## Aggregate Files:

In [23]:
results = pd.concat([results_log, results_knn[0], results_svm[0], results_nb[0]
                    , results_dt[0], results_rf[0]
                    ])

results.to_csv("../final_data/combined_2020_results.csv")

In [24]:
results

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.990217,0.377226,0.987598,0.601112,0.984697,0.676506,0.965227,0.751128,LogisticRegression,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}"
0,0.841181,0.436439,0.878540,0.554629,0.896552,0.567732,0.885246,0.619926,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 20}"
0,0.997555,0.168944,0.975362,0.723480,1.000000,0.395471,1.000000,0.246002,LinearSVC,"{'C': 0.01, 'random_state': 0}"
0,0.359927,0.898551,0.383568,0.940405,0.363594,0.947432,0.355835,0.940139,GaussianNB,{'priors': None}
0,1.000000,0.484886,1.000000,0.747716,0.979186,0.837040,0.912357,0.947519,DecisionTreeClassifier,"{'criterion': 'gini', 'max_depth': 1, 'min_sam..."
0,0.990741,0.221532,1.000000,0.394517,0.993761,0.450869,0.979428,0.448954,RandomForestClassifier,"{'n_estimators': 600, 'criterion': 'entropy', ..."
